In [2]:
%pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.7 MB/s eta 0:00:00


In [3]:
%pip install datasets
%pip install openai-whisper
%pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.1/153.1 kB 22.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multipl

In [3]:
from datasets import load_dataset,Audio
# from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load
import whisper
def evaluateModel(path):
    common_voice = load_dataset("mozilla-foundation/common_voice_11_0", "zh-HK", split="test")
    common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
    common_voice = common_voice.select(range(1000))
    model=whisper.load_model(path)

    # processor = WhisperProcessor.from_pretrained(path)
    # model = WhisperForConditionalGeneration.from_pretrained(path).to("cuda")



    def map_to_pred(batch):
        audio = batch["audio"]
        # input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
        # batch["reference"] = processor.tokenizer._normalize(batch['sentence'])
        batch["reference"]=batch['sentence']

        # with torch.no_grad():
        #     predicted_ids = model.generate(input_features.to("cuda"))[0]
        # transcription = processor.decode(predicted_ids)
        # batch["prediction"] = processor.tokenizer._normalize(transcription)
        batch["prediction"] = model.transcribe(audio["path"])["text"]
        return batch

    result = common_voice.map(map_to_pred)

    wer = load("wer")
    print(100 * wer.compute(references=result["reference"], predictions=result["prediction"]))


In [5]:
evaluateModel("tiny")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 8423it [00:00, 72905.81it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 21310it [00:00, 125853.10it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4289it [00:00, 88979.64it/s]
100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 118MiB/s]
Parameter 'function'=<function evaluateModel.<locals>.map_to_pred at 0x78d8a00ed3f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

222.6546906187625


In [6]:
evaluateModel("base")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 64.8MiB/s]


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

159.3812375249501


In [7]:
evaluateModel("small")

100%|████████████████████████████████████████| 461M/461M [00:04<00:00, 103MiB/s]


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

119.66067864271457


In [8]:
evaluateModel("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 105MiB/s]


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

96.9061876247505


In [9]:
evaluateModel("large")

100%|█████████████████████████████████████| 2.88G/2.88G [01:09<00:00, 44.7MiB/s]


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

101.99600798403195


In [4]:
evaluateModel("large-v2")

Parameter 'function'=<function evaluateModel.<locals>.map_to_pred at 0x7a0473e85d80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

104.99001996007983


In [ ]:
evaluateModel("large-v3")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]